# now do some ML using the web scraped data

In [273]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [274]:
from player_data_expansion import get_all_expanded_player_fight_data

In [275]:
# AWS integration getting file data
df_init = pd.read_csv("fight_data.csv").fillna(pd.NA)
df_init.head()

,Date,fighter1,fighter1_result,method_or_round,fighter2,Venue,Undercard fights,link,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage
0,"Saturday, 3 May 2025",Canelo Alvarez,Winner,"UD, Round 12",William Scull,"The Venue, Riyadh, Saudi Arabia",Jaime Munguia Beat Bruno Surace (UD 12) \nMar...,https://box.live/fights/canelo-vs-scull/,62,2,2,23,0,0,175,182.0,0,0
1,"Saturday, 3 May 2025",Jukiya Iimura,Winner,"UD, Round 12",Esneth Domingo,"Korakuen Hall, Tokyo, Japan",<NA>,https://box.live/fights/iimura-vs-domingo/,8,1,0,21,2,0,164,160.0,0,0
2,"Friday, 2 May 2025",Ryan Garcia,Loser,"UD, Round 12",Rolando ‘Rolly’ Romero,"Times Square, New York, NY, United States",Devin Haney Beat Jose Ramirez (UD 12) \nTeofi...,https://box.live/fights/garcia-vs-romero/,24,1,0,16,2,0,178,173.0,83,81
3,"Thursday, 1 May 2025",Kevin Lerena,Winner,"TKO, Round 3",Serhiy Radchenko,"Sunbet Arena, Pretoria, South Africa",<NA>,https://box.live/fights/lerena-vs-radchenko/,30,3,0,11,7,0,185,186.0,47,45
4,"Sunday, 27 April 2025",Marlon Tapales,Winner,"RTD, Round 3",Jon Jon Jet,"General Santos City, Cotabato del Sur, Philipp...",<NA>,https://box.live/fights/tapales-vs-jet/,39,4,0,15,1,1,163,166.0,0,0


### start cleaning

In [276]:
df_essentials = df_init.drop(columns=['link','Venue','Date','Undercard fights','fighter1','fighter2'])
df_essentials.head(2)

,fighter1_result,method_or_round,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage
0,Winner,"UD, Round 12",62,2,2,23,0,0,175,182.0,0,0
1,Winner,"UD, Round 12",8,1,0,21,2,0,164,160.0,0,0


In [277]:
df_staged_cleaning = df_essentials.copy()
df_staged_cleaning.head(2)

,fighter1_result,method_or_round,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage
0,Winner,"UD, Round 12",62,2,2,23,0,0,175,182.0,0,0
1,Winner,"UD, Round 12",8,1,0,21,2,0,164,160.0,0,0


In [278]:
df_staged_cleaning['method_or_round'] = df_staged_cleaning['method_or_round'].str.split(' ')

In [279]:
last_rounds = [new_list[2] for new_list in df_staged_cleaning['method_or_round']][:]
df_staged_cleaning['round'] = last_rounds

df_staged_cleaning.head()

,fighter1_result,method_or_round,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,round
0,Winner,"[UD,, Round, 12]",62,2,2,23,0,0,175,182.0,0,0,12
1,Winner,"[UD,, Round, 12]",8,1,0,21,2,0,164,160.0,0,0,12
2,Loser,"[UD,, Round, 12]",24,1,0,16,2,0,178,173.0,83,81,12
3,Winner,"[TKO,, Round, 3]",30,3,0,11,7,0,185,186.0,47,45,3
4,Winner,"[RTD,, Round, 3]",39,4,0,15,1,1,163,166.0,0,0,3


In [280]:
df_staged_cleaning = df_staged_cleaning.drop(columns='method_or_round')
df_staged_cleaning.head()

,fighter1_result,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,round
0,Winner,62,2,2,23,0,0,175,182.0,0,0,12
1,Winner,8,1,0,21,2,0,164,160.0,0,0,12
2,Loser,24,1,0,16,2,0,178,173.0,83,81,12
3,Winner,30,3,0,11,7,0,185,186.0,47,45,3
4,Winner,39,4,0,15,1,1,163,166.0,0,0,3


### encoding

In [580]:
df_encoded = df_staged_cleaning.copy()

In [581]:
def encoder(value) :
    if value == 'Winner' :
        return 1
    elif value=='Loser' :
        return 0
    else : return 2

In [582]:
df_encoded['fighter1_result'] = df_encoded['fighter1_result'].apply(encoder)
df_encoded['fighter1_result'] = df_encoded['fighter1_result'].astype('category')
df_encoded['fighter1_result'].head(10)

0    1
1    1
2    0
3    1
4    1
5    1
6    1
7    1
8    1
9    2
Name: fighter1_result, dtype: category
Categories (3, int64): [0, 1, 2]

In [583]:
# results_encoded = pd.get_dummies(df_encoded['fighter1_result'], drop_first=True) # 1 = Loss, 2 = Win, none = draw
# results_encoded.head()

In [584]:
# df_encoded = pd.concat([df_encoded, results_encoded], axis=1)
# df_encoded.head()

In [585]:
# df_encoded.drop(columns='fighter1_result', inplace=True)

In [586]:
# df_encoded.columns = ['fighter1_record_wins',  'fighter1_record_draws',
#        'fighter1_record_losses',   'fighter2_record_wins',
#         'fighter2_record_draws', 'fighter2_record_losses',
#            'fighter1_height_cm',     'fighter2_height_cm',
#        'fighter1_ko_percentage', 'fighter2_ko_percentage',
#                         'round',                        'fighter1_Loss',
#                               'fighter1_Win']

In [587]:
df_encoded.shape

(12, 12)

In [588]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   fighter1_result         12 non-null     category
 1   fighter1_record_wins    12 non-null     int64   
 2   fighter1_record_draws   12 non-null     int64   
 3   fighter1_record_losses  12 non-null     int64   
 4   fighter2_record_wins    12 non-null     int64   
 5   fighter2_record_draws   12 non-null     int64   
 6   fighter2_record_losses  12 non-null     int64   
 7   fighter1_height_cm      12 non-null     int64   
 8   fighter2_height_cm      11 non-null     float64 
 9   fighter1_ko_percentage  12 non-null     int64   
 10  fighter2_ko_percentage  12 non-null     int64   
 11  round                   12 non-null     object  
dtypes: category(1), float64(1), int64(9), object(1)
memory usage: 1.3+ KB


In [589]:
df_encoded = df_encoded.infer_objects()
df_encoded['fighter1_record_wins'] = df_encoded['fighter1_record_wins'].astype('Int64')
df_encoded['fighter1_record_draws'] = df_encoded['fighter1_record_draws'].astype('Int64')
df_encoded['fighter1_record_losses'] = df_encoded['fighter1_record_losses'].astype('Int64')
df_encoded['fighter2_record_wins'] = df_encoded['fighter2_record_wins'].astype('Int64')
df_encoded['fighter2_record_draws'] = df_encoded['fighter2_record_draws'].astype('Int64')
df_encoded['fighter2_record_losses'] = df_encoded['fighter1_record_losses'].astype('Int64')
df_encoded['fighter2_height_cm'] = df_encoded['fighter2_height_cm'].astype('Int64')
df_encoded['fighter1_ko_percentage'] = df_encoded['fighter1_ko_percentage'].astype('Int64')
df_encoded['fighter2_ko_percentage'] = df_encoded['fighter2_ko_percentage'].astype('Int64')
df_encoded['round'] = df_encoded['round'].astype('Int64')

In [590]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   fighter1_result         12 non-null     category
 1   fighter1_record_wins    12 non-null     Int64   
 2   fighter1_record_draws   12 non-null     Int64   
 3   fighter1_record_losses  12 non-null     Int64   
 4   fighter2_record_wins    12 non-null     Int64   
 5   fighter2_record_draws   12 non-null     Int64   
 6   fighter2_record_losses  12 non-null     Int64   
 7   fighter1_height_cm      12 non-null     int64   
 8   fighter2_height_cm      11 non-null     Int64   
 9   fighter1_ko_percentage  12 non-null     Int64   
 10  fighter2_ko_percentage  12 non-null     Int64   
 11  round                   12 non-null     Int64   
dtypes: Int64(10), category(1), int64(1)
memory usage: 1.4 KB


In [591]:
df_encoded.describe()

,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,round
count,12.0,12.0,12.0,12.0,12.0,12.0,12.000000,11.0,12.0,12.0,12.0
mean,24.416667,1.583333,0.333333,19.666667,2.666667,0.333333,176.666667,174.636364,29.166667,24.083333,8.5
std,16.434212,1.78164,0.651339,4.618802,2.498484,0.651339,7.900901,9.383738,37.201743,31.154916,3.965763
min,6.0,0.0,0.0,11.0,0.0,0.0,163.000000,160.0,0.0,0.0,2.0
25%,13.25,0.0,0.0,15.75,0.75,0.0,175.000000,169.5,0.0,0.0,6.0
50%,20.5,1.0,0.0,21.5,2.0,0.0,175.500000,175.0,0.0,0.0,10.0
75%,34.25,3.0,0.25,23.0,4.5,0.25,180.750000,182.5,65.75,48.5,12.0
max,62.0,5.0,2.0,26.0,7.0,2.0,191.000000,186.0,83.0,81.0,12.0


## TRAIN MODEL - XGBoost First

In [592]:
from xgboost import XGBClassifier

In [593]:
xgbc = XGBClassifier(
    objective='multi:softmax',
    random_state=42,
    num_class=3,
)

In [594]:
from sklearn.model_selection import GridSearchCV
# grid = GridSearchCV(xgbc, )

In [595]:
from sklearn.model_selection import train_test_split

In [603]:
X = df_encoded.drop(columns=['fighter1_result'])
y = df_encoded['fighter1_result']
y.unique()

[1, 0, 2]
Categories (3, int64): [0, 1, 2]

In [609]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [610]:
xgbc.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=3, ...)

In [611]:
xgbc_preds = xgbc.predict(X_test)

In [612]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [613]:
print(accuracy_score(y_test, xgbc_preds))

0.75


In [614]:
xgbc_preds

array([1, 1, 1, 1])